In [1]:
import psycopg2
from config import load_config
import pandas as pd
import folium
import os
from sklearn.cluster import DBSCAN
from geopy.distance import geodesic
import numpy as np

In [19]:
def capturar_data_mapa(conn, table_name, linha):
    cur = conn.cursor()
    all_data = [] 
    try:
        fetch_query = f'''
        SELECT latitude::double precision, longitude::double precision
        FROM {table_name}
        WHERE linha='{linha}'
        '''
        cur.execute(fetch_query)
        rows = cur.fetchall()
        all_data.extend(rows)
    except Exception as e:
            print(f"Erro ao executar a query na tabela {table_name}: {e}")
            conn.rollback()
    else:
         conn.commit()
    cur.close()
    return all_data

In [20]:
def limpar_dados_tabela(table_name, conn):
    cur = conn.cursor()
    cur.execute(f"""
        ALTER TABLE {table_name} ADD COLUMN IF NOT EXISTS hour INTEGER;
        UPDATE {table_name} SET hour = EXTRACT(HOUR FROM TO_TIMESTAMP(datahora / 1000));
        
        CREATE TABLE {table_name}_filter AS
        SELECT *
        FROM {table_name}
        WHERE hour BETWEEN 8 AND 22;
        
        ALTER TABLE {table_name} DROP COLUMN hour;
        ALTER TABLE {table_name}_filter DROP COLUMN hour;
    """)
    cur.close()
    conn.commit()

In [21]:
def adicionar_geom(table_name, conn):
    cur = conn.cursor()
    try:
        cur.execute(f"""
            ALTER TABLE {table_name}_filter ADD COLUMN IF NOT EXISTS geom geography(Point, 4326);
        """)
        cur.execute(f"""
            UPDATE {table_name}_filter SET geom = ST_SetSRID(ST_MakePoint(longitude::double precision, latitude::double precision), 4326);
        """)
        print(f"Coluna geom criada e preenchida na tabela {table_name}_filter.")
    except Exception as e:
        print(f"Erro ao atualizar a tabela {table_name}_filter: {e}")
        conn.rollback()
    else:
         conn.commit()
    cur.close()

In [22]:
def filtra_linhas(table_name, linhas_de_interesse, conn):
    cur = conn.cursor()
    temp_table_name = table_name + '_temp'
    linhas_str = ', '.join([f"'{linha}'" for linha in linhas_de_interesse])
    try:
        # Criar tabela temporária filtrada
        cur.execute(f"""
            CREATE TEMP TABLE {temp_table_name} AS
            SELECT *
            FROM {table_name}_filter
            WHERE linha IN ({linhas_str});
        """)
        print(f"Tabela temporária {temp_table_name} criada com sucesso.")
        conn.commit()
    except Exception as e:
        print(f"Erro ao criar a tabela temporária {temp_table_name}: {e}")
        conn.rollback()
    cur.close()

In [23]:
def sobrescreve_tabelas(table_name, conn):
    cur = conn.cursor()
    temp_table_name = table_name + '_temp'
    try:
        # Excluir a tabela original
        cur.execute(f"DROP TABLE {table_name}_filter;")
        # Renomear a tabela temporária para o nome original
        cur.execute(f"ALTER TABLE {temp_table_name} RENAME TO {table_name}_filter;")
        print(f"Tabela {table_name}_filter sobrescrita com sucesso.")
        conn.commit()
    except Exception as e:
        print(f"Erro ao sobrescrever a tabela {table_name}: {e}")
        conn.rollback()
    cur.close()

In [93]:
def margem_erro(coord1, coord2, radius=100):
    return geodesic(coord1, coord2).meters <= radius

In [24]:
def connect(config):
    """ Connect to the PostgreSQL database server """
    try:
        # connecting to the PostgreSQL server
        with psycopg2.connect(**config) as conn:
            print('Connected to the PostgreSQL server.')
            return conn
    except (psycopg2.DatabaseError, Exception) as error:
        print(error)


In [25]:
config = load_config()
conn = connect(config)

Connected to the PostgreSQL server.


In [28]:
linhas_de_interesse = ['483', '864', '639', '3', '309', '774', '629', '371', '397', '100', '838', '315', '624', '388', '918', '665', '328', '497', '878', '355', '138', '606', '457', '550', '803', '917', '638', '2336', '399', '298', '867', '553', '565', '422', '756', '186012003', '292', '554', '634', '232', '415', '2803', '324', '852', '557', '759', '343', '779', '905', '108']

In [29]:
table_names = ['dia_0105', 'dia_0205', 'dia_0305', 'dia_0405', 'dia_0505', 'dia_0605', 'dia_0705', 'dia_0805', 'dia_0905', 'dia_1005', 'dia_2504', 'dia_2604', 'dia_2704', 'dia_2804', 'dia_2904', 'dia_3004']

In [30]:
for table in table_names:
    limpar_dados_tabela(table, conn)

In [32]:
for table in table_names:
    adicionar_geom(table, conn)

Coluna geom criada e preenchida na tabela dia_0105_filter.
Coluna geom criada e preenchida na tabela dia_0205_filter.
Coluna geom criada e preenchida na tabela dia_0305_filter.
Coluna geom criada e preenchida na tabela dia_0405_filter.
Coluna geom criada e preenchida na tabela dia_0505_filter.
Coluna geom criada e preenchida na tabela dia_0605_filter.
Coluna geom criada e preenchida na tabela dia_0705_filter.
Coluna geom criada e preenchida na tabela dia_0805_filter.
Coluna geom criada e preenchida na tabela dia_0905_filter.
Coluna geom criada e preenchida na tabela dia_1005_filter.
Coluna geom criada e preenchida na tabela dia_2504_filter.
Coluna geom criada e preenchida na tabela dia_2604_filter.
Coluna geom criada e preenchida na tabela dia_2704_filter.
Coluna geom criada e preenchida na tabela dia_2804_filter.
Coluna geom criada e preenchida na tabela dia_2904_filter.
Coluna geom criada e preenchida na tabela dia_3004_filter.


In [33]:
for table in table_names:
    filtra_linhas(table, linhas_de_interesse, conn)
    sobrescreve_tabelas(table, conn)

Tabela temporária dia_0105_temp criada com sucesso.
Tabela dia_0105_filter sobrescrita com sucesso.
Tabela temporária dia_0205_temp criada com sucesso.
Tabela dia_0205_filter sobrescrita com sucesso.
Tabela temporária dia_0305_temp criada com sucesso.
Tabela dia_0305_filter sobrescrita com sucesso.
Tabela temporária dia_0405_temp criada com sucesso.
Tabela dia_0405_filter sobrescrita com sucesso.
Tabela temporária dia_0505_temp criada com sucesso.
Tabela dia_0505_filter sobrescrita com sucesso.
Tabela temporária dia_0605_temp criada com sucesso.
Tabela dia_0605_filter sobrescrita com sucesso.
Tabela temporária dia_0705_temp criada com sucesso.
Tabela dia_0705_filter sobrescrita com sucesso.
Tabela temporária dia_0805_temp criada com sucesso.
Tabela dia_0805_filter sobrescrita com sucesso.
Tabela temporária dia_0905_temp criada com sucesso.
Tabela dia_0905_filter sobrescrita com sucesso.
Tabela temporária dia_1005_temp criada com sucesso.
Tabela dia_1005_filter sobrescrita com sucesso.


In [39]:
"""for linha in linhas_de_interesse:
    data = capturar_data_mapa(conn, 'dia_2704_filter', linha)
    if data:
        avg_latitude = sum([d[0] for d in data]) / len(data)
        avg_longitude = sum([d[1] for d in data]) / len(data)
        m = folium.Map(location=[avg_latitude, avg_longitude], zoom_start=12)
        # Adicione os pontos ao mapa
        for lat, lon in data:
            folium.CircleMarker(location=[lat, lon], radius=1, color='blue').add_to(m)
        output_path = os.path.join(f'trajetos/output_map_{linha}.html')
        m.save(output_path)
        print(f"Mapa salvo em {output_path}")"""

Mapa salvo em trajetos/output_map_483.html
Mapa salvo em trajetos/output_map_864.html
Mapa salvo em trajetos/output_map_639.html
Mapa salvo em trajetos/output_map_3.html
Mapa salvo em trajetos/output_map_309.html
Mapa salvo em trajetos/output_map_774.html
Mapa salvo em trajetos/output_map_629.html
Mapa salvo em trajetos/output_map_371.html
Mapa salvo em trajetos/output_map_397.html
Mapa salvo em trajetos/output_map_100.html
Mapa salvo em trajetos/output_map_838.html
Mapa salvo em trajetos/output_map_315.html
Mapa salvo em trajetos/output_map_624.html
Mapa salvo em trajetos/output_map_388.html
Mapa salvo em trajetos/output_map_918.html
Mapa salvo em trajetos/output_map_665.html
Mapa salvo em trajetos/output_map_328.html
Mapa salvo em trajetos/output_map_497.html
Mapa salvo em trajetos/output_map_878.html
Mapa salvo em trajetos/output_map_355.html
Mapa salvo em trajetos/output_map_606.html
Mapa salvo em trajetos/output_map_457.html
Mapa salvo em trajetos/output_map_550.html
Mapa salvo em

In [87]:
query = """
WITH ordered_points AS (
    SELECT
        ordem,
        linha,
        geom,
        TO_TIMESTAMP(datahora / 1000) AS datahora_ts,
        LAG(TO_TIMESTAMP(datahora / 1000)) OVER (PARTITION BY ordem ORDER BY TO_TIMESTAMP(datahora / 1000)) AS prev_datahora_ts,
        LAG(geom) OVER (PARTITION BY ordem ORDER BY TO_TIMESTAMP(datahora / 1000)) AS prev_geom,
        ROW_NUMBER() OVER (PARTITION BY ordem ORDER BY TO_TIMESTAMP(datahora / 1000)) AS rn
    FROM dia_2704_filter
),
same_position_periods AS (
    SELECT
        ordem,
        linha,
        geom,
        datahora_ts,
        prev_datahora_ts,
        EXTRACT(EPOCH FROM (datahora_ts - prev_datahora_ts)) AS duration,
        CASE 
            WHEN ST_DWithin(geom, prev_geom, 15) THEN 1
            ELSE 0
        END AS is_stationary,
        ROW_NUMBER() OVER (PARTITION BY ordem ORDER BY datahora_ts) - 
        ROW_NUMBER() OVER (PARTITION BY ordem, CASE WHEN ST_DWithin(geom, prev_geom, 15) THEN 1 ELSE 0 END ORDER BY datahora_ts) AS grp
    FROM ordered_points
),
stationary_groups AS (
    SELECT
        ordem,
        linha,
        geom,
        datahora_ts,
        prev_datahora_ts,
        duration,
        is_stationary,
        grp
    FROM same_position_periods
    WHERE is_stationary = 1
)
SELECT
    ordem,
    linha,
    MIN(prev_datahora_ts) AS start_time,
    MAX(datahora_ts) AS end_time,
    SUM(duration) AS total_duration,
    ST_X(geom::geometry) AS longitude,
    ST_Y(geom::geometry) AS latitude,
    grp
FROM stationary_groups
GROUP BY ordem, linha, grp, geom
HAVING SUM(duration) >= 600; -- 10 minutes in seconds
"""

In [88]:
df = pd.read_sql(query, conn)

/tmp/ipykernel_4808/1168034203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [89]:
df_unique = df.drop_duplicates(subset=['grp'])

In [94]:
pontos_de_parada = {}

In [95]:
for linha in linhas_de_interesse:
    pontos_de_parada[linha]=[]
    for _, row in df_unique.loc[df_unique['linha'] == linha].iterrows():
        lat, lon = row['latitude'], row['longitude']
        ponto = (lat, lon)
        encontrado = False
        if row['total_duration'] > 24000:
            continue
        for pontos in pontos_de_parada[linha]:
            if margem_erro(pontos, ponto):
                encontrado = True
                break
        if not encontrado:
            pontos_de_parada[linha].append(ponto)


In [96]:
for linha in linhas_de_interesse:
    tamanho = len(pontos_de_parada[linha])
    print(f"A {linha} tem {tamanho} paradas.")

A 483 tem 4 paradas.
A 864 tem 2 paradas.
A 639 tem 5 paradas.
A 3 tem 1 paradas.
A 309 tem 2 paradas.
A 774 tem 2 paradas.
A 629 tem 3 paradas.
A 371 tem 1 paradas.
A 397 tem 3 paradas.
A 100 tem 3 paradas.
A 838 tem 3 paradas.
A 315 tem 8 paradas.
A 624 tem 2 paradas.
A 388 tem 2 paradas.
A 918 tem 3 paradas.
A 665 tem 6 paradas.
A 328 tem 3 paradas.
A 497 tem 2 paradas.
A 878 tem 2 paradas.
A 355 tem 3 paradas.
A 138 tem 0 paradas.
A 606 tem 3 paradas.
A 457 tem 4 paradas.
A 550 tem 3 paradas.
A 803 tem 3 paradas.
A 917 tem 3 paradas.
A 638 tem 2 paradas.
A 2336 tem 3 paradas.
A 399 tem 4 paradas.
A 298 tem 3 paradas.
A 867 tem 3 paradas.
A 553 tem 2 paradas.
A 565 tem 2 paradas.
A 422 tem 2 paradas.
A 756 tem 4 paradas.
A 186012003 tem 0 paradas.
A 292 tem 5 paradas.
A 554 tem 2 paradas.
A 634 tem 2 paradas.
A 232 tem 3 paradas.
A 415 tem 4 paradas.
A 2803 tem 4 paradas.
A 324 tem 3 paradas.
A 852 tem 2 paradas.
A 557 tem 4 paradas.
A 759 tem 2 paradas.
A 343 tem 2 paradas.
A 779 t